### 后处理方法试验

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import warnings
from sklearn.metrics import cohen_kappa_score, confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder
import os
import itertools

warnings.filterwarnings('ignore')
os.getcwd() 
os.chdir('./data')


### 读取数据
y_Q3_3 = pd.read_csv('./y_train_3/y_Q3_3.csv')
y_Q4_3 = pd.read_csv('./y_train_3/y_Q4_3.csv')

aum_fils = os.listdir('x_train/aum_train/')+os.listdir('x_test/aum_test/')
aum = []
for f in aum_fils:
    print(f)
    mon = int((f.split('.')[0]).split('_')[-1].replace('m', ''))
    if mon>=7:
        tmp = pd.read_csv('x_train/aum_train/'+f)
        tmp['mon'] = mon
    else:
        tmp = pd.read_csv('x_test/aum_test/'+f)
        tmp['mon'] = mon+12
    aum.append(tmp)
aum = pd.concat(aum, axis=0, ignore_index=True)

behavior_fils = os.listdir('x_train/behavior_train/')+os.listdir('x_test/behavior_test/')
behavior = []
for f in behavior_fils:
    print(f)
    mon = int((f.split('.')[0]).split('_')[-1].replace('m', ''))
    if mon>=7:
        tmp = pd.read_csv('x_train/behavior_train/'+f)
        tmp['mon'] = mon
    else:
        tmp = pd.read_csv('x_test/behavior_test/'+f)
        tmp['mon'] = mon+12
    behavior.append(tmp)
behavior = pd.concat(behavior, axis=0, ignore_index=True)

event_fils = os.listdir('x_train/big_event_train/')+os.listdir('x_test/big_event_test/')
event = []
for f in event_fils:
    print(f)
    season = int((f.split('.')[0]).split('_')[-1].replace('Q', ''))
    if season>=3:
        tmp = pd.read_csv('x_train/big_event_train/'+f)
        tmp['season'] = season
    else:
        tmp = pd.read_csv('x_test/big_event_test/'+f)
        tmp['season'] = season + 4
    
    event.append(tmp)
event = pd.concat(event, axis=0, ignore_index=True)

cunkuan_fils = os.listdir('x_train/cunkuan_train/')+os.listdir('x_test/cunkuan_test/')
cunkuan = []
for f in cunkuan_fils:
    print(f)
    mon = int((f.split('.')[0]).split('_')[-1].replace('m', ''))
    if mon>=7:
        tmp = pd.read_csv('x_train/cunkuan_train/'+f)
        tmp['mon'] = mon
    else:
        tmp = pd.read_csv('x_test/cunkuan_test/'+f)
        tmp['mon'] = mon+12
    cunkuan.append(tmp)
cunkuan = pd.concat(cunkuan, axis=0, ignore_index=True)

cust_avli_Q3 = pd.read_csv('./x_train/cust_avli_Q3.csv')
cust_avli_Q4 = pd.read_csv('./x_train/cust_avli_Q4.csv')
cust_info_Q3 = pd.read_csv('x_train/cust_info_q3.csv')
cust_info_Q4 = pd.read_csv('x_train/cust_info_q4.csv')
cust_avli_Q1 = pd.read_csv('x_test/cust_avli_Q1.csv')
cust_info_Q1 = pd.read_csv('x_test/cust_info_q1.csv')

aum_m10.csv
aum_m11.csv
aum_m12.csv
aum_m7.csv
aum_m8.csv
aum_m9.csv
aum_m1.csv
aum_m2.csv
aum_m3.csv
behavior_m10.csv
behavior_m11.csv
behavior_m12.csv
behavior_m7.csv
behavior_m8.csv
behavior_m9.csv
behavior_m1.csv
behavior_m2.csv
behavior_m3.csv
big_event_Q3.csv
big_event_Q4.csv
big_event_Q1.csv
cunkuan_m10.csv
cunkuan_m11.csv
cunkuan_m12.csv
cunkuan_m7.csv
cunkuan_m8.csv
cunkuan_m9.csv
cunkuan_m1.csv
cunkuan_m2.csv
cunkuan_m3.csv


## 特征工程

第一组特征很自然的想到用户历史的label，例如在预测季度4的用户时，使用用户在季度3的label作为特征。可以简单看到这个特征的kappa值可以达到0.238+。

In [2]:
train = y_Q4_3.copy()
y_Q3_3 = y_Q3_3.rename(columns={'label': 'bef_label'})
train = train.merge(y_Q3_3, on=['cust_no'], how='left').copy()

test = cust_avli_Q1.copy()
y_Q4_3 = y_Q4_3.rename(columns={'label': 'bef_label'})
test = test.merge(y_Q4_3, on=['cust_no'], how='left')
test

,cust_no,bef_label
0,0x3b9b4615,0.0
1,0x3b9ae61b,1.0
2,0x3b9add69,0.0
3,0x3b9b3601,0.0
4,0x3b9b2599,0.0
...,...,...
76717,0xb2d69017,0.0
76718,0xb2d68153,1.0
76719,0xb2d5bba1,1.0
76720,0xb2d61b9b,1.0


### 用户信息
比较重要的是用户等级

In [3]:
# 根据EDA去掉一部分的列
cust_info_Q4.drop(columns = ['I7','I8','I9','I12','I15','I17','I18','I19','I20'], inplace=True)
cust_info_Q3.drop(columns = ['I7','I8','I9','I12','I15','I17','I18','I19','I20'], inplace=True)
cust_info_Q1.drop(columns = ['I7','I8','I9','I12','I15','I17','I18','I19','I20'], inplace=True)
train = train.merge(cust_info_Q4, on=['cust_no'], how='left')
train = train.merge(cust_info_Q3[['cust_no','I3']], on=['cust_no'], how='left')
test = test.merge(cust_info_Q1, on=['cust_no'], how='left')
test = test.merge(cust_info_Q4[['cust_no','I3']], on=['cust_no'], how='left')
test

,cust_no,bef_label,I1,I2,I3_x,I4,I5,I6,I10,I11,I13,I14,I16,I3_y
0,0x3b9b4615,0.0,女性,27,黄金,0.0,服务性工作人员,0,NaN,0.0,NaN,NaN,1.0,黄金
1,0x3b9ae61b,1.0,女性,45,黄金,1.0,NaN,1,NaN,0.0,NaN,NaN,1.0,黄金
2,0x3b9add69,0.0,男性,75,黄金,0.0,NaN,0,NaN,0.0,NaN,NaN,1.0,黄金
3,0x3b9b3601,0.0,女性,49,黄金,0.0,NaN,0,NaN,0.0,NaN,NaN,1.0,黄金
4,0x3b9b2599,0.0,女性,60,黄金,0.0,不便分类的其他从业人员,0,NaN,0.0,NaN,NaN,1.0,黄金
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76717,0xb2d69017,0.0,女性,19,普通客户,0.0,商业工作人员,0,NaN,0.0,NaN,NaN,0.0,普通客户
76718,0xb2d68153,1.0,女性,67,黄金,0.0,服务性工作人员,0,NaN,0.0,NaN,NaN,1.0,黄金
76719,0xb2d5bba1,1.0,女性,59,白金,0.0,不便分类的其他从业人员,0,NaN,0.0,NaN,NaN,1.0,黄金
76720,0xb2d61b9b,1.0,男性,73,普通客户,0.0,农、林、牧、渔、水利业生产人员,0,NaN,0.0,NaN,NaN,0.0,普通客户


In [4]:
### 对年龄进行分箱
def get_age(x):
    if x<18:
        return 1
    elif x<30:
        return 2
    elif x<40:
        return 3 
    elif x<50:
        return 4    
    elif x<60:
        return 5  
    elif x<80:
        return 6
    else:
        return 7  

train['I2'] = train['I2'].apply(lambda x: get_age(x))
test['I2'] = test['I2'].apply(lambda x: get_age(x))

In [5]:
### 其他特征编码
for col in [f for f in train.select_dtypes('object').columns if f not in ['label', 'cust_no']]:
    print(col)
    train[col].fillna(train[col].mode()[0], inplace=True)### 缺失值处理·
    test[col].fillna(train[col].mode()[0], inplace=True)### 缺失值处理·
    le = LabelEncoder()
    le.fit(pd.concat([train[[col]], test[[col]]], axis=0, ignore_index=True))
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

I1
I3_x
I5
I10
I13
I14
I3_y


### 用户行为特征

这题最重要的应该是用户行为相关的数据，下面我们开始做一些简单的操作：
1. 用户当季度存款（cunkuan）的mean、max、min、std、sum、last的统计
2. 用户当季度最后一个月的aum数据
3. 用户当季度最后一个月的behavior数据
4. 用户当季度的event的特征，这里大多数都是时间，所以用该季度月末的后一天做时间差特征

### 特征工程1
#### 1. 用户当季度存款(cunkuan)的mean、max、min、std、sum、last、skew、kurtosis的统计

In [ ]:
def drop_correlated_col(df, cutoff):
    # 筛选高度相关特征
    def filter_corr(corr, cutoff=0.9):
        cols = []
        for i,j in feature_group:
            if corr.loc[i, j] > cutoff:
                print(i,j,corr.loc[i, j])
                i_avg = corr[i][corr[i] != 1].mean()
                j_avg = corr[j][corr[j] != 1].mean()
                if i_avg >= j_avg:
                    cols.append(i)
                else:
                    cols.append(j)
        return set(cols)

    corr = df.corr()
    feature_group = list(itertools.combinations(corr.columns, 2))
    drop_cols = filter_corr(corr, cutoff)
    print(list(drop_cols))
    df.drop(list(drop_cols),inplace = True, axis = 1)
    return df


def statistics_feature(df, agg_stat, acutoff=0.96)
    # 数据处理
    group_df3 = cunkuan[(cunkuan['mon']<=9)&(cunkuan['mon']>=7)].groupby(['cust_no']).agg(agg_stat)
    group_df3.columns = [f[0]+'_'+f[1] for f in group_df3.columns]
    group_df3.reset_index(inplace=True)
    group_df3['season'] = 3

    group_df4 = cunkuan[(cunkuan['mon']<=12)&(cunkuan['mon']>=10)].groupby(['cust_no']).agg(agg_stat)
    col = [f[0]+'_'+f[1] for f in group_df4.columns]
    group_df4.columns = [f[0]+'_'+f[1] for f in group_df4.columns]
    group_df4.reset_index(inplace=True)
    group_df4['season'] = 4

    group_df1 = cunkuan[(cunkuan['mon']<=15)&(cunkuan['mon']>=13)].groupby(['cust_no']).agg(agg_stat)
    group_df1.columns = [f[0]+'_'+f[1] for f in group_df1.columns]
    group_df1.reset_index(inplace=True)
    group_df1['season'] = 5
                           
    cunkuan_stat = pd.concat([pd.concat([group_df3, group_df4], axis=0, ignore_index=True), group_df1], axis=0, ignore_index=True)
    del group_df3, group_df4, group_df1      
    
    # 剔除高度相关特征
    cunkuan_stat = drop_correlated_col(cunkuan_stat, cutoff)
    
    # 将特征合并进去         
    train, test = merge_feat(train, test, cunkuan_stat)
    
    return train, test

### 特征工程2
#### 2. 用户存款(cunkuan)的季内和季间的变化

In [53]:
def get_Fluction_Feature(df_fea, train, test, f):
    df_fea = df_fea[~df_fea['cust_no'].isnull()] #去除id为空的行
    stat = pd.DataFrame(df_fea[['cust_no']].drop_duplicates())#去除重复值
    for i in range(7,16):
        tmp = df_fea[(df_fea['mon']==i)][['cust_no', f]].copy()
        stat = stat.merge(tmp, on=['cust_no'], how='left')
        print(stat.shape)
    stat.fillna(value=0, inplace=True)
    stat.columns =['cust_no'] + [f + '_' + str(i) for i in range(7, 16)]
    # 季度内波动
    stat[f+'_3s'] = stat[f +'_9'] - stat[f +'_7']
    stat[f+'_4s'] = stat[f +'_12'] - stat[f +'_10']
    stat[f+'_5s'] = stat[f +'_15'] - stat[f +'_13']
    # 季度间波动
    stat[f+'_34s'] = stat[f +'_12'] - stat[f +'_9']
    stat[f+'_45s'] = stat[f +'_15'] - stat[f +'_12']


    tmp = stat[['cust_no', f+'_9', f+'_12', f+'_3s', f+'_4s', f+'_34s']].copy()
    tmp.columns = ['cust_no'] + [f+'_'+str(i) for i in range(1,6)]
    train = train.merge(tmp, on=['cust_no'], how='left')
    del tmp

    tmp = stat[['cust_no',f+'_12', f+'_15', f+'_4s', f+'_5s',f+'_45s']].copy()
    tmp.columns = ['cust_no'] + [f+'_'+str(i) for i in range(1,6)]
    test = test.merge(tmp, on=['cust_no'], how='left')
    del tmp
    return train, test


In [54]:
train, test =  get_Fluction_Feature(cunkuan, train, test, 'C1')

(254839, 2)
(254839, 3)
(254839, 4)
(254839, 5)
(254839, 6)
(254839, 7)
(254839, 8)
(254839, 9)
(254839, 10)


In [7]:
cunkuan = cunkuan[~cunkuan['cust_no'].isnull()] #去除id为空的行
cunkuan_stat = pd.DataFrame(cunkuan[['cust_no']].drop_duplicates())#去除重复值

for i in range(7,16):
    tmp = cunkuan[(cunkuan['mon']==i)][['cust_no', 'C1']].copy()
    cunkuan_stat = cunkuan_stat.merge(tmp, on=['cust_no'], how='left')
    print(cunkuan_stat.shape)
cunkuan_stat.columns = ['cust_no'] + ['C1_'+ str(i)for i in range(7,16)]
cunkuan_stat.fillna(0, inplace=True)
# 静态
cunkuan_stat['C1'] =  cunkuan_stat['C1_9']  
cunkuan_stat['C2'] =  cunkuan_stat['C1_12']  
cunkuan_stat['C3'] =  cunkuan_stat['C1_15']  
# 季度内
cunkuan_stat['C4'] =  cunkuan_stat['C1_9'] - cunkuan_stat['C1_7']   
cunkuan_stat['C5'] = cunkuan_stat['C1_12'] - cunkuan_stat['C1_10']
cunkuan_stat['C6'] = cunkuan_stat['C1_15'] - cunkuan_stat['C1_13']
# 季度间
cunkuan_stat['C7'] = cunkuan_stat['C1_12'] - cunkuan_stat['C1_9'] 
cunkuan_stat['C8'] = cunkuan_stat['C1_15'] - cunkuan_stat['C1_12'] 

cunkuan_stat

(254839, 2)
(254839, 3)
(254839, 4)
(254839, 5)
(254839, 6)
(254839, 7)
(254839, 8)
(254839, 9)
(254839, 10)


,cust_no,C1_7,C1_8,C1_9,C1_10,C1_11,C1_12,C1_13,C1_14,C1_15,C1,C2,C3,C4,C5,C6,C7,C8
0,0xb2d3ad96,332174.06,332005.48,331893.29,231097.73,231097.73,125081.24,125021.24,125021.24,103040.20,331893.29,125081.24,103040.20,-280.77,-106016.49,-21981.04,-206812.05,-22041.04
1,0xb2d40f1c,5.33,5.33,5.34,5.34,5.34,5.35,5.35,5.35,5.36,5.34,5.35,5.36,0.01,0.01,0.01,0.01,0.01
2,0xb2d6bb22,7260.50,10060.90,10466.58,13808.58,17117.90,16023.22,23473.22,18249.22,17566.58,10466.58,16023.22,17566.58,3206.08,2214.64,-5906.64,5556.64,1543.36
3,0xb2d6bb1f,40032.65,40006.70,40945.59,42322.89,40086.64,43102.03,40240.51,46855.37,45245.03,40945.59,43102.03,45245.03,912.94,779.14,5004.52,2156.44,2143.00
4,0xb2d3d20b,120007.40,120007.40,140007.62,140007.62,140007.62,140230.85,40000.18,40000.18,40007.27,140007.62,140230.85,40007.27,20000.22,223.23,7.09,223.23,-100223.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254834,0xb2dbcd91,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
254835,0xb2dbd307,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,2.00,0.00,0.00,2.00,0.00,2.00
254836,0xb2dbe90c,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4355.28,0.00,0.00,4355.28,0.00,0.00,4355.28,0.00,4355.28
254837,0xb2dbe960,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,918.15,0.00,0.00,918.15,0.00,0.00,918.15,0.00,918.15


In [80]:
import itertools

def drop_correlated_col(df, cutoff):
    def filter_corr(corr, cutoff=0.7):
        cols = []
        for i,j in feature_group:
            if corr.loc[i, j] > cutoff:
                print(i,j,corr.loc[i, j])
                i_avg = corr[i][corr[i] != 1].mean()
                j_avg = corr[j][corr[j] != 1].mean()
                if i_avg >= j_avg:
                    cols.append(i)
                else:
                    cols.append(j)
        return set(cols)

    corr = df.corr()
    feature_group = list(itertools.combinations(corr.columns, 2))
    drop_cols = filter_corr(corr, cutoff)
    print(list(drop_cols))
    df.drop(list(drop_cols),inplace = True, axis = 1)
    return df

def merge_feat(train, test, df):
    tmp = df[df['season']==3].copy()
    del tmp['season']
    tmp.columns = ['cust_no'] +  [f+'_1' for f in tmp.columns[1:]]
    col_1 = [f for f in tmp.columns[1:]]
    train = train.merge(tmp, on=['cust_no'], how='left')
    train[col_1].fillna(value=0, inplace=True)


    tmp = df[df['season']==4].copy()
    del tmp['season']
    tmp.columns = ['cust_no'] +  [f+'_2' for f in tmp.columns[1:]]
    col_2 = [f for f in tmp.columns[1:]]
    train = train.merge(tmp, on=['cust_no'], how='left')
    train[col_2].fillna(value=0, inplace=True)


    for i,j  in zip(col_1, col_2):
        train[j+'_'+i] = train[j] - train[i]

    tmp = df[df['season']==4].copy()
    del tmp['season']
    tmp.columns = ['cust_no'] +  [f+'_1' for f in tmp.columns[1:]]
    col_1 = [f for f in tmp.columns[1:]]
    test = test.merge(tmp, on=['cust_no'], how='left')
    test[col_1].fillna(value=0, inplace=True)


    tmp = df[df['season']==5].copy()
    del tmp['season']
    tmp.columns = ['cust_no'] +  [f+'_2' for f in tmp.columns[1:]]
    col_2 = [f for f in tmp.columns[1:]]
    test = test.merge(tmp, on=['cust_no'], how='left')
    test[col_2].fillna(value=0, inplace=True)

    for i,j  in zip(col_1, col_2):
        test[j+'_'+i] = test[j] - test[i]
        
    return train, test

cunkuan_stat = pd.read_pickle('cunkuan_stat.pkl')
   
cunkuan_stat = drop_correlated_col(cunkuan_stat, cutoff = 0.96)

train, test = merge_feat(train, test, cunkuan_stat)

In [45]:
print(train.isnull().sum() / len(train))
# tmp = cunkuan_stat[cunkuan_stat['season']==3].copy()
# del tmp['season']
# tmp.columns = ['cust_no'] +  [f+'_1' for f in tmp.columns[1:]]
# col_1 = [f for f in tmp.columns[1:]]
# train = train.merge(tmp, on=['cust_no'], how='left')
# train[col_1].fillna(value=0, inplace=True)


# tmp = cunkuan_stat[cunkuan_stat['season']==4].copy()
# del tmp['season']
# tmp.columns = ['cust_no'] +  [f+'_2' for f in tmp.columns[1:]]
# col_2 = [f for f in tmp.columns[1:]]
# train = train.merge(tmp, on=['cust_no'], how='left')
# train[col_2].fillna(value=0, inplace=True)


# for i,j  in zip(col_1, col_2):
#     train[j+'_'+i] = train[j] - train[i]

cust_no                0.000000
label                  0.000000
bef_label              0.180819
I1                     0.000000
I2                     0.000000
                         ...   
C3_min_2_C3_min_1      0.113247
C3_std_2_C3_std_1      0.157043
C3_sum_2_C3_sum_1      0.113247
C3_last_2_C3_last_1    0.113247
C3_skew_2_C3_skew_1    0.193265
Length: 69, dtype: float64


In [12]:
train.shape, test.shape

((76170, 53), (76722, 52))

In [60]:
train.columns

Index(['cust_no', 'label', 'bef_label', 'I1', 'I2', 'I3_x', 'I4', 'I5', 'I6',
       'I10', 'I11', 'I13', 'I14', 'I16', 'I3_y', 'C1_min_1', 'C1_std_1',
       'C1_skew_1', 'C2_min_1', 'C2_std_1', 'C2_skew_1', 'C3_max_1',
       'C3_min_1', 'C3_std_1', 'C3_sum_1', 'C3_skew_1', 'C1_min_2', 'C1_std_2',
       'C1_skew_2', 'C2_min_2', 'C2_std_2', 'C2_skew_2', 'C3_max_2',
       'C3_min_2', 'C3_std_2', 'C3_sum_2', 'C3_skew_2', 'C1_min_2_C1_min_1',
       'C1_std_2_C1_std_1', 'C1_skew_2_C1_skew_1', 'C2_min_2_C2_min_1',
       'C2_std_2_C2_std_1', 'C2_skew_2_C2_skew_1', 'C3_max_2_C3_max_1',
       'C3_min_2_C3_min_1', 'C3_std_2_C3_std_1', 'C3_sum_2_C3_sum_1',
       'C3_skew_2_C3_skew_1', 'C1_1', 'C1_2', 'C1_3', 'C1_4', 'C1_5',
       'X_sum_1', 'X_sum_2', 'X_sum_3', 'X_sum_4', 'X_sum_5'],
      dtype='object')

#### 资产数据(aum)

In [55]:
def get_Fluction_Feature(df_fea, train, test, f):
#     f = 'X3'
    df_fea = df_fea[~df_fea['cust_no'].isnull()] #去除id为空的行
    stat = pd.DataFrame(df_fea[['cust_no']].drop_duplicates())#去除重复值
    for i in range(7,16):
        tmp = df_fea[(df_fea['mon']==i)][['cust_no', f]].copy()
        stat = stat.merge(tmp, on=['cust_no'], how='left')
        print(stat.shape)
        
    stat.fillna(value=0, inplace=True)#用0填充
    stat.columns =['cust_no'] + [f + '_' + str(i) for i in range(7, 16)]
    # 季度内波动
    stat[f+'_3s'] = stat[f +'_9'] - stat[f +'_7']
    stat[f+'_4s'] = stat[f +'_12'] - stat[f +'_10']
    stat[f+'_5s'] = stat[f +'_15'] - stat[f +'_13']
    # 季度间波动
    stat[f+'_34s'] = stat[f +'_12'] - stat[f +'_9']
    stat[f+'_45s'] = stat[f +'_15'] - stat[f +'_12']


    tmp = stat[['cust_no', f+'_9', f+'_12', f+'_3s', f+'_4s', f+'_34s']].copy()
    tmp.columns = ['cust_no'] + [f+'_'+str(i) for i in range(1,6)]
    train = train.merge(tmp, on=['cust_no'], how='left')
    del tmp

    tmp = stat[['cust_no',f+'_12', f+'_15', f+'_4s', f+'_5s',f+'_45s']].copy()
    tmp.columns = ['cust_no'] + [f+'_'+str(i) for i in range(1,6)]
    test = test.merge(tmp, on=['cust_no'], how='left')
    del tmp
    return train, test


In [56]:
X_cols = [f for f in aum.columns if f.startswith('X')]
aum['X_sum'] = aum[X_cols].sum(axis=1)
train, test =  get_Fluction_Feature(aum, train, test, 'X_sum')

(659624, 2)
(659624, 3)
(659624, 4)
(659624, 5)
(659624, 6)
(659624, 7)
(659624, 8)
(659624, 9)
(659624, 10)


In [66]:
aum_stat = pd.read_pickle('aum_stat.pkl')
aum_stat = drop_correlated_col(aum_stat, cutoff = 0.96)
train, test = merge_feat(train, test, aum_stat)
print(train.shape, test.shape)

(76170, 196) (76722, 195)


#### 3. 用户当季度最后一个月的行为数据(behavior)

#### 4. 用户当季度的事件特征(event)，这里大多数都是时间，所以用该季度月末的后一天做时间差特征。

In [24]:

behavior['B5-B3'] = behavior['B5'] - behavior['B3']
behavior['B4-B2'] = behavior['B4'] - behavior['B2']
behavior['B3/B2'] = behavior['B3'] / behavior['B2']
behavior['B5/B4'] = behavior['B5'] / behavior['B4']
behavior['B4/B2'] = behavior['B4'] / behavior['B2']
behavior['B5/B3'] = behavior['B5'] / behavior['B3']
behavior['B6_hour'] =  pd.to_datetime(behavior['B6'] ).dt.hour
behavior.loc[((behavior['mon']==9)),'B6'] =  (pd.to_datetime('2019-10-01 00:00:00') -pd.to_datetime(behavior.loc[((behavior['mon']==9)),'B6'])).dt.days
behavior.loc[((behavior['mon']==12)),'B6'] =  (pd.to_datetime('2020-01-01 00:00:00')-pd.to_datetime(behavior.loc[((behavior['mon']==12)),'B6'])).dt.days
behavior.loc[((behavior['mon']==15)),'B6'] =  (pd.to_datetime('2020-04-01 00:00:00') -pd.to_datetime(behavior.loc[((behavior['mon']==15)),'B6'])).dt.days

# (pd.to_datetime('2019-10-01 00:00:00') -pd.to_datetime(behavior.loc[((behavior['mon']==9)),'B6'])).dt.days

In [ ]:
agg_stat = {'B5-B3': ['max', 'min', 'std', 'sum', 'last', 'skew'],
            'B4-B2': ['max', 'min', 'std', 'sum', 'last', 'skew'],
            'B3/B2': ['max', 'min', 'std', 'sum', 'last', 'skew'],
            'B5/B4': ['max', 'min', 'std', 'sum', 'last', 'skew'],
            'B4/B2': ['max', 'min', 'std', 'sum', 'last', 'skew'],
            'B5/B3': ['max', 'min', 'std', 'sum', 'last', 'skew'],
            'B1': ['max', 'min', 'std', 'sum', 'last', 'skew'],
            'B2': ['max', 'min', 'std', 'sum', 'last', 'skew'],
            'B3': ['max', 'min', 'std', 'sum', 'last', 'skew'],
            'B4': ['max', 'min', 'std', 'sum', 'last', 'skew'],
            'B5': ['max', 'min', 'std', 'sum', 'last', 'skew'],
             'B6': ['max'], 
            'B6_hour': ['max'], 
            'B7': ['max'],        
           }



In [74]:
behavior_stat = pd.read_pickle('behavior_stat.pkl')
behavior_stat = drop_correlated_col(behavior_stat, cutoff = 0.96)
train, test = merge_feat(train, test, behavior_stat)
print(train.shape, test.shape)

B3/B2_min B3/B2_last 0.9714778417386742
B5/B4_max B5/B4_sum 0.9612153008223376
B5/B4_max B5/B4_last 0.9678604292715175
B5/B4_min B5/B4_last 0.9690609554203113
B5/B3_max B5/B3_min 0.9968893511806203
B5/B3_max B5/B3_std 0.99752284106521
B5/B3_max B5/B3_sum 0.9999999985612688
B5/B3_max B5/B3_last 0.9969164322151617
B5/B3_min B5/B3_sum 0.9968893437084174
B5/B3_min B5/B3_last 0.9999728105501697
B5/B3_std B5/B3_sum 0.997522396716309
B5/B3_sum B5/B3_last 0.9969164276516074
B3_max B3_std 0.9601763869847368
B4_max B4_std 0.9651829078903473
['B5/B4_sum', 'B5/B3_last', 'B4_max', 'B3_max', 'B3/B2_last', 'B5/B3_sum', 'B5/B3_max', 'B5/B4_max', 'B5/B4_last']
(76170, 376) (76722, 375)


### 事件相关特征

In [84]:
# event_feat = pd.read_pickle('event_feat_engineering1.pkl')
# behavior_stat = drop_correlated_col(event_feat, cutoff = 0.96)
train = train.merge(behavior_stat.loc[behavior_stat['flag']==1], on=['cust_no'], how='left')
del train['flag']
test = test.merge(behavior_stat.loc[behavior_stat['flag']==0], on=['cust_no'], how='left')
del test['flag']
print(train.shape, test.shape)

(76170, 392) (76722, 391)


### 去掉和标签几乎完全不相关的特征

In [85]:
drop_cols = train.corr()['label'][abs(train.corr()['label'])<0.005].index
test.drop(drop_cols,axis=1,inplace=True)
train.drop(drop_cols,axis=1,inplace=True)
print(train.shape, test.shape)

(76170, 318) (76722, 317)


### 加入权重信息

In [60]:
train['weight'] = train['label'].map({0:1.03,1:0.58,-1:1})

### 指定类别标签

In [76]:
cate_cols =['bef_label'] + [f for f in train.columns if f.startswith('I') and f not in  ['I11'] ] 
cate_cols

['bef_label',
 'I1',
 'I2',
 'I3_x',
 'I4',
 'I5',
 'I6',
 'I10',
 'I13',
 'I14',
 'I16',
 'I3_y']

## 模型训练

以上就构成了我们baseline的基础特征，下面开始训练模型。这里采用的是Lightgbm进行5折的多分类，早停直接使用kappa值。因为训练多分类时，目标值的最小值得是0，所以我们对原始label做+1的处理（记得提交的时候要改回来）。

In [65]:
def kappa(preds, train_data):
    y_true = train_data.label
    preds = np.argmax(preds.reshape(3, -1), axis=0)
    score = cohen_kappa_score(y_true, preds)
    return 'kappa', score, True

def LGB_classfication_model(train, target, test, k):
    feats = [f for f in train.columns if f not in ['cust_no', 'label', 'weight']]
    print('Current num of features:', len(feats))
    folds = KFold(n_splits=k, shuffle=False, random_state=2020)
    oof_preds = np.zeros(train.shape[0])
    oof_probs = np.zeros((train.shape[0], 3))
    output_preds = []
    feature_importance_df = pd.DataFrame()
    offline_score = []
    for i, (train_index, test_index) in enumerate(folds.split(train, target)):
        train_y, test_y = target[train_index], target[test_index]
        train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
        print(train_X.shape, test_X.shape)
        dtrain = lgb.Dataset(train_X,
                             label=train_y,
                             categorical_feature=cate_cols, 
                            )
        dval = lgb.Dataset(test_X,
                           label=test_y,
                          categorical_feature=cate_cols, )
        parameters = {
            'learning_rate': 0.05,
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'metric': 'None',
            'num_leaves': 63,
            'num_class': 3,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'min_data_in_leaf': 20,
            'verbose': -1,
            'nthread': 24
        }
        lgb_model = lgb.train(
            parameters,
            dtrain,
            num_boost_round=5000,
            valid_sets=[dval],
            early_stopping_rounds=100,
            verbose_eval=100,
            feval=kappa, 
        )
        oof_probs[test_index] = lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration)
        oof_preds[test_index] = np.argmax(lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration), axis=1)
        offline_score.append(lgb_model.best_score['valid_0']['kappa'])
            

        output_preds.append(lgb_model.predict(test[feats], num_iteration=lgb_model.best_iteration))
        # feature importance
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = lgb_model.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
    print('OOF-MEAN-KAPPA score:%.6f, OOF-STD:%.6f' % (np.mean(offline_score), np.std(offline_score)))
    print('feature importance:')
    print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))
    print('confusion matrix:')
    print(confusion_matrix(target, oof_preds))
    print('classfication report:')
    print(classification_report(target, oof_preds))

    return output_preds, oof_probs, np.mean(offline_score)

In [66]:
target = train['label'] + 1
lgb_preds, lgb_oof, lgb_score = LGB_classfication_model(train, target, test, 5)

Current num of features: 362
(60936, 362) (15234, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.470796
[200]	valid_0's kappa: 0.473298
Early stopping, best iteration is:
[194]	valid_0's kappa: 0.474366
(60936, 362) (15234, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.465506
[200]	valid_0's kappa: 0.466089
Early stopping, best iteration is:
[122]	valid_0's kappa: 0.467985
(60936, 362) (15234, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.459165
[200]	valid_0's kappa: 0.461454
[300]	valid_0's kappa: 0.462316
Early stopping, best iteration is:
[269]	valid_0's kappa: 0.464943
(60936, 362) (15234, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.449488
[200]	valid_0's kappa: 0.449385
Early stopping, best iteration is:
[136]	valid_0's kappa: 0.453221
(60936, 362) (15234, 362)
Training until validation scores don'

In [67]:
print(cohen_kappa_score(train['label'] + 1, np.argmax(lgb_oof, axis=1)))

0.46526508048401805


## 后处理

### 使用sp.optimize.minimize

In [68]:
from sklearn.metrics import cohen_kappa_score
from functools import partial
import scipy as sp

def kappa_loss(weight, y_hat, y):
    y_hat = weight*y_hat
    loss = cohen_kappa_score(y, np.argmax(y_hat, axis=1))
    return -loss

def get_weights(y_hat, y):
    size = np.unique(y).size
    loss_partial = partial(kappa_loss, y_hat=y_hat, y=y)
    initial_weights = [1. for _ in range(size)]
    weights_ = sp.optimize.minimize(loss_partial, initial_weights, method='Nelder-Mead')
    return weights_['x']



In [69]:
weights = get_weights(lgb_oof, train['label'] + 1)
print('权重为',weights)
print(cohen_kappa_score(train['label'] + 1, np.argmax(lgb_oof @ np.diag(weights), axis=1)))


权重为 [1.45716273 1.11638232 0.64310668]
0.48918081744161745


### 贝叶斯搜索

In [57]:
from bayes_opt import BayesianOptimization
def get_bayes_weights(y_hat, y):
    def kappa_calc(x1, x2, x3):
        pred_y = np.array([x1, x2, x3])*y_hat
        loss = cohen_kappa_score(y, np.argmax(pred_y, axis=1))
        return loss

    optimizer = BayesianOptimization(
        f=kappa_calc,
        pbounds={
            "x1": (0, 3),
            "x2": (0, 3),
            "x3": (0, 3),
        },
        random_state=1,
        verbose=0
    )
    optimizer.maximize()
    weights_ = optimizer.max['params']
    weights_ = np.array([weights_['x1'], weights_['x2'], weights_['x3']])
    return weights_

In [59]:
bayes_weights =  get_bayes_weights(lgb_oof, train['label'] + 1)

print(cohen_kappa_score(train['label'] + 1, np.argmax(lgb_oof @ np.diag(bayes_weights), axis=1)))


0.48466115374052543


In [61]:
def kappa(preds, train_data):
    y_true = train_data.label
    preds = np.argmax(preds.reshape(3, -1), axis=0)
    score = cohen_kappa_score(y_true, preds)
    return 'kappa', score, True

def LGB_classfication_model(train, target, test, k):
    feats = [f for f in train.columns if f not in ['cust_no', 'label', 'weight']]
    print('Current num of features:', len(feats))
    folds = KFold(n_splits=k, shuffle=False, random_state=2020)
    oof_preds = np.zeros(train.shape[0])
    oof_probs = np.zeros((train.shape[0], 3))
    output_preds = []
    feature_importance_df = pd.DataFrame()
    offline_score = []
    for i, (train_index, test_index) in enumerate(folds.split(train, target)):
        train_y, test_y = target[train_index], target[test_index]
        train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
        train_weight, valid_weight = train['weight'][train_index], train['weight'][test_index]
        print(train_X.shape, test_X.shape)
        dtrain = lgb.Dataset(train_X,
                             label=train_y,
                             categorical_feature=cate_cols, 
                             weight=train_weight.values.flatten(order='F'),
                            )
        dval = lgb.Dataset(test_X,
                           label=test_y,
                          categorical_feature=cate_cols, )
        parameters = {
            'learning_rate': 0.05,
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'metric': 'None',
            'num_leaves': 63,
            'num_class': 3,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'min_data_in_leaf': 20,
            'verbose': -1,
            'nthread': 24
        }
        lgb_model = lgb.train(
            parameters,
            dtrain,
            num_boost_round=5000,
            valid_sets=[dval],
            early_stopping_rounds=100,
            verbose_eval=100,
            feval=kappa, 
        )
        oof_probs[test_index] = lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration)
        oof_preds[test_index] = np.argmax(lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration), axis=1)
        offline_score.append(lgb_model.best_score['valid_0']['kappa'])
            

        output_preds.append(lgb_model.predict(test[feats], num_iteration=lgb_model.best_iteration))
        # feature importance
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = lgb_model.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('OOF-MEAN-KAPPA score:%.6f, OOF-STD:%.6f' % (np.mean(offline_score), np.std(offline_score)))
    print('feature importance:')
    print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))
    print('confusion matrix:')
    print(confusion_matrix(target, oof_preds))
    print('classfication report:')
    print(classification_report(target, oof_preds))

    return output_preds, oof_probs, np.mean(offline_score)

In [62]:
target = train['label'] + 1
lgb_preds, lgb_oof, lgb_score = LGB_classfication_model(train, target, test, 5)

Current num of features: 362
(60936, 362) (15234, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.493764
[200]	valid_0's kappa: 0.498521
Early stopping, best iteration is:
[188]	valid_0's kappa: 0.499305
(60936, 362) (15234, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.484062
[200]	valid_0's kappa: 0.488291
Early stopping, best iteration is:
[162]	valid_0's kappa: 0.48893
(60936, 362) (15234, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.472903
[200]	valid_0's kappa: 0.477027
[300]	valid_0's kappa: 0.475891
Early stopping, best iteration is:
[213]	valid_0's kappa: 0.479163
(60936, 362) (15234, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.483272
[200]	valid_0's kappa: 0.481256
Early stopping, best iteration is:
[131]	valid_0's kappa: 0.485338
(60936, 362) (15234, 362)
Training until validation scores don't

In [63]:
print(cohen_kappa_score(train['label'] + 1, np.argmax(lgb_oof, axis=1)))


0.4898882468809378


In [43]:
sub_df = test[['cust_no']].copy()
sub_df['label'] = np.argmax(np.mean(lgb_preds, axis=0), axis=1) - 1
sub_df['label'].value_counts(normalize=True)

 1    0.633495
 0    0.225307
-1    0.141198
Name: label, dtype: float64

In [45]:
def kappa(preds, train_data):
    y_true = train_data.label
    preds = np.argmax(preds.reshape(3, -1), axis=0)
    score = cohen_kappa_score(y_true, preds)
    return 'kappa', score, True

def LGB_classfication_model(train, target, test, k):
    feats = [f for f in train.columns if f not in ['cust_no', 'label', 'weight']]
    print('Current num of features:', len(feats))
    folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)
    oof_preds = np.zeros(train.shape[0])
    oof_probs = np.zeros((train.shape[0], 3))
    output_preds = []
    feature_importance_df = pd.DataFrame()
    offline_score = []
    for i, (train_index, test_index) in enumerate(folds.split(train, target)):
        train_y, test_y = target[train_index], target[test_index]
        train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
        train_weight, valid_weight = train['weight'][train_index], train['weight'][test_index]
        print(train_X.shape, test_X.shape)
        dtrain = lgb.Dataset(train_X,
                             label=train_y,
                             categorical_feature=cate_cols, 
                             weight=train_weight.values.flatten(order='F'),
                            )
        dval = lgb.Dataset(test_X,
                           label=test_y,
                          categorical_feature=cate_cols, )
        parameters = {
            'learning_rate': 0.05,
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'metric': 'None',
            'num_leaves': 63,
            'num_class': 3,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'min_data_in_leaf': 20,
            'verbose': -1,
            'nthread': 24
        }
        lgb_model = lgb.train(
            parameters,
            dtrain,
            num_boost_round=5000,
            valid_sets=[dval],
            early_stopping_rounds=100,
            verbose_eval=100,
            feval=kappa, 
        )
        oof_probs[test_index] = lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration)
        oof_preds[test_index] = np.argmax(lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration), axis=1)
        offline_score.append(lgb_model.best_score['valid_0']['kappa'])
            

        output_preds.append(lgb_model.predict(test[feats], num_iteration=lgb_model.best_iteration))
        # feature importance
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = lgb_model.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('OOF-MEAN-KAPPA score:%.6f, OOF-STD:%.6f' % (np.mean(offline_score), np.std(offline_score)))
    print('feature importance:')
    print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))
    print('confusion matrix:')
    print(confusion_matrix(target, oof_preds))
    print('classfication report:')
    print(classification_report(target, oof_preds))

    return output_preds, oof_probs, np.mean(offline_score)

In [49]:
target = train['label'] + 1
lgb_preds, lgb_oof, lgb_score = LGB_classfication_model(train, target, test, 5)

Current num of features: 362
(60934, 362) (15236, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.485067
[200]	valid_0's kappa: 0.482683
Early stopping, best iteration is:
[134]	valid_0's kappa: 0.487408
(60935, 362) (15235, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.491094
Early stopping, best iteration is:
[60]	valid_0's kappa: 0.492062
(60937, 362) (15233, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.494069
Early stopping, best iteration is:
[79]	valid_0's kappa: 0.49547
(60937, 362) (15233, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.482474
[200]	valid_0's kappa: 0.485149
[300]	valid_0's kappa: 0.48635
Early stopping, best iteration is:
[243]	valid_0's kappa: 0.488465
(60937, 362) (15233, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.48066
[200]	valid_

In [200]:
# sub_df[['prba1', 'prba2', 'prba3']] = np.mean(lgb_preds, axis=0)
sub_df.to_csv('lgb26_prab.csv', index=False)
sub_df

,cust_no,label,prba1,prba2,prba3
0,0x3b9b4615,1,0.064529,0.426921,0.508550
1,0x3b9ae61b,1,0.023218,0.114226,0.862556
2,0x3b9add69,1,0.044682,0.194048,0.761270
3,0x3b9b3601,0,0.014320,0.724767,0.260913
4,0x3b9b2599,0,0.013054,0.508893,0.478053
...,...,...,...,...,...
76717,0xb2d69017,1,0.119785,0.265532,0.614684
76718,0xb2d68153,1,0.010656,0.163321,0.826024
76719,0xb2d5bba1,1,0.073333,0.206951,0.719716
76720,0xb2d61b9b,1,0.056048,0.134226,0.809726


In [50]:
sub_df = test[['cust_no']].copy()
sub_df['label'] = np.argmax(np.mean(lgb_preds, axis=0), axis=1) - 1
sub_df['label'].value_counts(normalize=True)

 1    0.630210
 0    0.236308
-1    0.133482
Name: label, dtype: float64

In [51]:
sub_df.to_csv('lgb26.csv', index=False)

In [86]:
def kappa(preds, train_data):
    y_true = train_data.label
    preds = np.argmax(preds.reshape(3, -1), axis=0)
    score = cohen_kappa_score(y_true, preds)
    return 'kappa', score, True

def LGB_classfication_model(train, target, test, k):
    feats = [f for f in train.columns if f not in ['cust_no', 'label', 'weight']]
    print('Current num of features:', len(feats))
    folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)
    oof_preds = np.zeros(train.shape[0])
    oof_probs = np.zeros((train.shape[0], 3))
    output_preds = []
    feature_importance_df = pd.DataFrame()
    offline_score = []
    for i, (train_index, test_index) in enumerate(folds.split(train, target)):
        train_y, test_y = target[train_index], target[test_index]
        train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
        train_weight, valid_weight = train['weight'][train_index], train['weight'][test_index]
        print(train_X.shape, test_X.shape)
        dtrain = lgb.Dataset(train_X,
                             label=train_y,
                             categorical_feature=cate_cols, 
                             weight=train_weight.values.flatten(order='F'),
                            )
        dval = lgb.Dataset(test_X,
                           label=test_y,
                          categorical_feature=cate_cols, )
        parameters = {
            'learning_rate': 0.05,
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'metric': 'None',
            'num_leaves': 63,
            'num_class': 3,
            'feature_fraction': 0.7,
            'bagging_fraction': 0.7,
            'min_data_in_leaf': 20,
            'verbose': -1,
            'nthread': 24
        }
        lgb_model = lgb.train(
            parameters,
            dtrain,
            num_boost_round=5000,
            valid_sets=[dval],
            early_stopping_rounds=100,
            verbose_eval=100,
            feval=kappa, 
        )
        oof_probs[test_index] = lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration)
        oof_preds[test_index] = np.argmax(lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration), axis=1)
        offline_score.append(lgb_model.best_score['valid_0']['kappa'])
            

        output_preds.append(lgb_model.predict(test[feats], num_iteration=lgb_model.best_iteration))
        # feature importance
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = lgb_model.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('OOF-MEAN-KAPPA score:%.6f, OOF-STD:%.6f' % (np.mean(offline_score), np.std(offline_score)))
    print('feature importance:')
    print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))
    print('confusion matrix:')
    print(confusion_matrix(target, oof_preds))
    print('classfication report:')
    print(classification_report(target, oof_preds))

    return output_preds, oof_probs, np.mean(offline_score)

In [87]:
target = train['label'] + 1
lgb_preds, lgb_oof, lgb_score = LGB_classfication_model(train, target, test, 5)

Current num of features: 362
(60934, 362) (15236, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.482662
[200]	valid_0's kappa: 0.483023
Early stopping, best iteration is:
[181]	valid_0's kappa: 0.487324
(60935, 362) (15235, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.492389
[200]	valid_0's kappa: 0.491433
[300]	valid_0's kappa: 0.489218
Early stopping, best iteration is:
[250]	valid_0's kappa: 0.493711
(60937, 362) (15233, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.491843
[200]	valid_0's kappa: 0.489416
Early stopping, best iteration is:
[106]	valid_0's kappa: 0.493458
(60937, 362) (15233, 362)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.479726
[200]	valid_0's kappa: 0.484878
[300]	valid_0's kappa: 0.481933
Early stopping, best iteration is:
[232]	valid_0's kappa: 0.485901
(60937, 362) (15233, 362)
Train

In [56]:
train['I3_x'] 

0        3
1        0
2        0
3        3
4        2
        ..
76165    1
76166    1
76167    3
76168    0
76169    1
Name: I3_x, Length: 76170, dtype: int64

In [71]:
train[(train['I3_x'] == 1)&(train['E1']==1)]['label'].value_counts()

1    62
Name: label, dtype: int64

In [70]:
train[(train['I3_x'] == 2)&(train['E1']==1)]['label'].value_counts()

1    11
Name: label, dtype: int64

In [72]:
test[(test['I3_x'] == 1)&(test['E1']==1)]

,cust_no,bef_label,I1,I2,I3_x,I5,I6,I11,I13,I14,...,E8,E9,E10,E12,E13,E14,E15,E16,E18,event_num
398,0xb2da9bae,NaN,1,2.0,0,7,0,0.0,2,5,...,3,3,1,3,3,1,20000.0,1,1,10
479,0xb2da8dd1,NaN,0,6.0,0,0,0,0.0,2,5,...,3,3,1,3,3,3,0.0,3,3,7
1436,0xb2daa201,NaN,1,3.0,0,1,0,0.0,4,4,...,3,3,1,3,3,3,0.0,3,3,7
1910,0xb2da8de8,NaN,1,7.0,3,0,0,0.0,2,5,...,3,3,1,3,3,3,0.0,3,3,7
2829,0xb2da9aee,NaN,1,3.0,0,4,0,0.0,2,5,...,3,3,1,3,3,3,0.0,3,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73847,0xb2da79a1,NaN,0,5.0,0,0,0,0.0,2,5,...,3,3,1,3,3,3,0.0,3,3,7
74661,0xb2da8cdb,NaN,1,4.0,0,7,0,0.0,2,5,...,3,3,1,3,3,3,0.0,3,3,6
75406,0xb2da74d2,NaN,1,6.0,0,0,0,0.0,2,5,...,3,3,1,3,3,3,0.0,3,3,6
75614,0xb2da7e9d,NaN,0,5.0,0,0,0,0.0,2,5,...,3,3,1,3,3,1,1.0,1,1,10


In [76]:
sub_df[(test['I3_x'] == 2)&(test['E1']==1)]

,cust_no,label
50701,0xb2da72e0,1
76562,0xb2daa150,1


In [61]:
train[(train['I3_x'] == 0)]['label'].value_counts()

 1    21556
-1     8507
 0     7477
Name: label, dtype: int64

In [56]:
target = train['label'] + 1
lgb_preds, lgb_oof, lgb_score = LGB_classfication_model(train, target, test, 5)

Current num of features: 309
(64488, 309) (16123, 309)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.540317
Early stopping, best iteration is:
[77]	valid_0's kappa: 0.542492
(64488, 309) (16123, 309)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.531217
[200]	valid_0's kappa: 0.532959
Early stopping, best iteration is:
[148]	valid_0's kappa: 0.534663
(64488, 309) (16123, 309)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.552588
[200]	valid_0's kappa: 0.556367
Early stopping, best iteration is:
[196]	valid_0's kappa: 0.557441
(64489, 309) (16122, 309)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.529648
[200]	valid_0's kappa: 0.528668
Early stopping, best iteration is:
[120]	valid_0's kappa: 0.531503
(64491, 309) (16120, 309)
Training until validation scores don't improve for 100 rounds
[100]	valid_0's kappa: 0.531798
[200]	va

In [237]:
import pandas as pd
# lgb1 = pd.read_csv("lgb1.csv")
# lgb2 = pd.read_csv("lgb2.csv")
# lgb3 = pd.read_csv("lgb3.csv")
# lgb4 = pd.read_csv("lgb4.csv")
# lgb5 = pd.read_csv("lgb5.csv")
# lgb6 = pd.read_csv("lgb6.csv")
# lgb7 = pd.read_csv("lgb7.csv")
# lgb8 = pd.read_csv("lgb8.csv")
# lgb9 = pd.read_csv("lgb9.csv")
# lgb10 = pd.read_csv("lgb10.csv")
# lgb11 = pd.read_csv("lgb11.csv")
# lgb13 = pd.read_csv("lgb13.csv")
# lgb14 = pd.read_csv("lgb14.csv")
# lgb21 = pd.read_csv("lgb21.csv")
# lgb22 = pd.read_csv("lgb22.csv")
# lgb23 = pd.read_csv("lgb23.csv")
# lgb24 = pd.read_csv("lgb24.csv")
# lgb24 = pd.read_csv("lgb24.csv")
lgb_485 = pd.read_csv("lgb26_prab.csv")
cnn_445 = pd.read_csv("prediction_cnn_0445.csv")
xgb_1 = pd.read_csv("xgb_1.csv")
xgb_1_prab = pd.read_csv("xgb_1_prab.csv")

In [186]:
lgb_485

,cust_no,label,prba1,prba2,prba3
0,0x3b9b4615,1,0.064529,0.426921,0.508550
1,0x3b9ae61b,1,0.023218,0.114226,0.862556
2,0x3b9add69,1,0.044682,0.194048,0.761270
3,0x3b9b3601,0,0.014320,0.724767,0.260913
4,0x3b9b2599,0,0.013054,0.508893,0.478053
...,...,...,...,...,...
76717,0xb2d69017,1,0.119785,0.265532,0.614684
76718,0xb2d68153,1,0.010656,0.163321,0.826024
76719,0xb2d5bba1,1,0.073333,0.206951,0.719716
76720,0xb2d61b9b,1,0.056048,0.134226,0.809726


array([[0.06945424, 0.44883579, 0.48170997],
       [0.02966919, 0.13753896, 0.83279184],
       [0.05742764, 0.22992203, 0.71265032],
       ...,
       [0.06750677, 0.20759096, 0.72490226],
       [0.06739202, 0.13232507, 0.8002829 ],
       [0.15136426, 0.56719672, 0.28143901]])

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [184]:
(lgb_485['cust_no']==cnn_445['cust_no']).sum(axis=0)

76722

In [194]:
ensemble_1 = pd.DataFrame(cnn_445['cust_no'])
ensemble_1['label'] =  np.argmax((0.75*lgb_485[['prba1', 'prba2', 'prba3']].values + 0.25*cnn_445[['pred_-1', 'pred_0', 'pred_1']].values), axis=1) - 1
ensemble_1

,cust_no,label
0,0x3b9b4615,1
1,0x3b9ae61b,1
2,0x3b9add69,1
3,0x3b9b3601,0
4,0x3b9b2599,1
...,...,...
76717,0xb2d69017,1
76718,0xb2d68153,1
76719,0xb2d5bba1,1
76720,0xb2d61b9b,1


In [197]:
ensemble_1.to_csv('ensemble_1.csv', index=False)
ensemble_1

,cust_no,label
0,0x3b9b4615,1
1,0x3b9ae61b,1
2,0x3b9add69,1
3,0x3b9b3601,0
4,0x3b9b2599,1
...,...,...
76717,0xb2d69017,1
76718,0xb2d68153,1
76719,0xb2d5bba1,1
76720,0xb2d61b9b,1


In [181]:
(cnn_445['label']).value_counts()

 1    48065
 0    15543
-1    13114
Name: label, dtype: int64

In [182]:
(lgb_485['label']).value_counts()

 1    48351
 0    18130
-1    10241
Name: label, dtype: int64

In [180]:
(cnn_445['label']==lgb_485['label']).value_counts()

True     65440
False    11282
Name: label, dtype: int64

In [195]:

(ensemble_1['label']==lgb_485['label']).value_counts()

True     74451
False     2271
Name: label, dtype: int64

In [201]:

(ensemble_1['label']==cnn_445['label']).value_counts()

True     67623
False     9099
Name: label, dtype: int64

In [90]:
(lgb26['label']==lgb22['label']).value_counts()

True     73813
False     2909
Name: label, dtype: int64

In [89]:
(lgb26['label']==lgb22['label']).value_counts()

True     73813
False     2909
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import StratifiedKFold
import catboost as cat
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
oof = np.zeros([len(x_train), 7])
predictions = np.zeros(test_data.shape[0])  

for fold_, (train_index, test_index) in enumerate(folds.split(x_train, y_train)):
    print("fold n°{}".format(fold_ + 1))
    train_x, test_x, train_y, test_y = x_train.iloc[train_index], x_train.iloc[test_index], y_train.iloc[
        train_index], y_train.iloc[test_index]

    cbt_model = cat.CatBoostClassifier(iterations=2500, learning_rate=0.057, max_depth=7, l2_leaf_reg=2, verbose=100,
                                       early_stopping_rounds=50,eval_metric='MAE',cat_features=L, loss_function='MultiClass',
                                     )
    cbt_model.fit(train_x, train_y, eval_set=(test_x, test_y))
    predictions += cbt_model.predict(test_data)/5

In [92]:
import catboost as cat
import xgboost as xgb

In [152]:
np.array([1,0]).shape

(2,)

In [159]:
score = cohen_kappa_score(np.array([1,0.5]), np.array([1,1]))
score

0.0

In [155]:
np.array([1,0]).shape

(2,)

In [161]:
 np.argmax(np.array([[1,0],[2,1]]), axis=1)

array([0, 0], dtype=int64)

In [209]:

 
def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings
 
def quadratic_weighted_kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Calculates the quadratic weighted kappa
    quadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
 
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))
 
    hist_rater_a = histogram(rater_a, min_rating, max_rating)
 
    hist_rater_b = histogram(rater_b, min_rating, max_rating)
 
    numerator = 0.0
    denominator = 0.0
 
    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items
 
    return 1.0 - numerator / denominator

def buildHistMat(preds, label):
    #get round preds and number of classes
    preds = np.round(preds)
    num_classes = len(np.unique(label))

    #build histogram matrix
    histMat = np.zeros((num_classes, num_classes))
    for i in range(num_classes):
        for j in range(num_classes):
            histMat[i,j] = np.sum(((preds==i+1)*(label==j+1)))

    histMat = histMat.astype(int)
    return histMat


# def kappaScore(preds, dtrain):
#     #retruning a negative score because xgb is minimizing what we want to maximise :)
#     label = np.array(dtrain.get_label())
#     label = label.astype(int)
#     preds = np.array(preds)
#     preds = np.clip(np.round(preds), np.min(label), np.max(label)).astype(int)
#     kappa = quadratic_weighted_kappa(preds, label)
#     return '-kappa', -kappa

def kappaScore(preds, dtrain):
    y_true = np.array(dtrain.get_label())
    
#     print(y_true[:10], preds[:10])
#     print(y_true.shape, preds.shape)
    preds = np.argmax(preds, axis=1)
#     preds = np.argmax(preds.reshape(3, -1), axis=0)
    score = cohen_kappa_score(y_true, preds)
    return '-kappa', -score

def customObj(preds, dtrain):
    #get labels
    label = dtrain.get_label().astype(int)

    #clip round & clip predictions
    preds = np.clip(np.round(preds), np.min(label), np.max(label)).astype(int)

    #convert prediction into index starting at 0 (for Histogram Matrix)
    idx = np.array(preds.astype(int)-1)

    #build histogram matrix
    histMat = buildHistMat(preds, label)

    #return a "kappa-weighted" squared error grad & a constant hess
    grad = histMat[idx, (label-1)]*(idx-(label-1))
    hess = np.ones(shape=preds.shape)
    return grad, hess

In [226]:


def XGB_classfication_model(train, target, test, k):
    feats = [f for f in train.columns if f not in ['cust_no', 'label', 'weight']]
    print('Current num of features:', len(feats))
    folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)
    oof_preds = np.zeros(train.shape[0])
    oof_probs = np.zeros((train.shape[0], 3))
    output_preds = []
    feature_importance_df = pd.DataFrame()
    offline_score = []
    for i, (train_index, test_index) in enumerate(folds.split(train, target)):
        train_y, test_y = target[train_index], target[test_index]
        train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
        train_weight, valid_weight = train['weight'][train_index], train['weight'][test_index]
        print(train_X.shape, test_X.shape)
#         dtrain = lgb.Dataset(train_X,
#                              label=train_y,
#                              categorical_feature=cate_cols, 
#                              weight=train_weight.values.flatten(order='F'),
#                             )
#         dval = lgb.Dataset(test_X,
#                            label=test_y,
#                           categorical_feature=cate_cols, )

        dtrain = xgb.DMatrix(train_X, label=train_y, weight=train_weight.values.flatten(order='F'))
        dval = xgb.DMatrix(test_X, label=test_y)
        watchlist = [(dtrain, 'train'),(dval, 'valid')]
        
        param = {
        'eta': 0.05,
        'max_depth': 8,
#         'objective':'multi:softmax',# 多分类的问题
         'objective': 'multi:softprob',# 多分类的问题
        'booster':'gbtree',
        'num_class':3,# 类别数，与 multisoftmax 并用
        'n_jobs':20,
        'gamma':0.2,# 用于控制是否后剪枝的参数,越大越保守
        'min_child_weight':1,
        'subsample':0.7,# 随机采样训练样本
        'colsample_bytree':0.7,
        'seed':2021# 随机种子

        }
    
        xgb_model = xgb.train(
            param, 
            dtrain, 
            num_boost_round =100, 
            evals = watchlist,
#             obj=customObj, 
            feval=kappaScore, 
            verbose_eval=True, 
            early_stopping_rounds=5)

        oof_probs[test_index] = xgb_model.predict((xgb.DMatrix(test_X[feats])))
        oof_preds[test_index] = np.argmax(oof_probs[test_index], axis=1)
#         offline_score.append(xgb_model.best_score['valid_0']['kappa'])
            

        output_preds.append(xgb_model.predict(xgb.DMatrix(test[feats])))
        # feature importance
#         fold_importance_df = pd.DataFrame()
#         fold_importance_df["feature"] = feats
#         fold_importance_df["importance"] = xgb_model.feature_importance(importance_type='gain')
#         fold_importance_df["fold"] = i + 1
#         feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
#     print('OOF-MEAN-KAPPA score:%.6f, OOF-STD:%.6f' % (np.mean(offline_score), np.std(offline_score)))
#     print('feature importance:')
#     print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(15))
#     print('confusion matrix:')
    print(target[:2], oof_preds[:2], target.shape, oof_preds.shape)
    print(confusion_matrix(target, oof_preds))
    print('classfication report:')
    print(classification_report(target, oof_preds))

    return output_preds, oof_probs, np.mean(offline_score)

In [227]:
target = train['label'] + 1
xgb_preds, xgb_oof, lgb_score = XGB_classfication_model(train, target, test, 5)

Current num of features: 362
(60934, 362) (15236, 362)
[0]	train-merror:0.300354	valid-merror:0.294303	train--kappa:-0.487816	valid--kappa:-0.434192
Multiple eval metrics have been passed: 'valid--kappa' will be used for early stopping.

Will train until valid--kappa hasn't improved in 5 rounds.
[1]	train-merror:0.29118	valid-merror:0.285377	train--kappa:-0.503275	valid--kappa:-0.448615
[2]	train-merror:0.287542	valid-merror:0.280192	train--kappa:-0.509639	valid--kappa:-0.453382
[3]	train-merror:0.286531	valid-merror:0.279142	train--kappa:-0.511321	valid--kappa:-0.456203
[4]	train-merror:0.283694	valid-merror:0.277369	train--kappa:-0.516096	valid--kappa:-0.458485
[5]	train-merror:0.283383	valid-merror:0.277304	train--kappa:-0.516527	valid--kappa:-0.459902
[6]	train-merror:0.281775	valid-merror:0.275466	train--kappa:-0.51949	valid--kappa:-0.4626
[7]	train-merror:0.279652	valid-merror:0.27481	train--kappa:-0.522975	valid--kappa:-0.463686
[8]	train-merror:0.27897	valid-merror:0.274678	tra

[41]	train-merror:0.256909	valid-merror:0.264163	train--kappa:-0.562139	valid--kappa:-0.485124
[42]	train-merror:0.256095	valid-merror:0.264623	train--kappa:-0.563543	valid--kappa:-0.484053
[43]	train-merror:0.25493	valid-merror:0.263901	train--kappa:-0.565519	valid--kappa:-0.485218
[44]	train-merror:0.254469	valid-merror:0.264163	train--kappa:-0.566284	valid--kappa:-0.484929
[45]	train-merror:0.253831	valid-merror:0.264492	train--kappa:-0.567432	valid--kappa:-0.48418
Stopping. Best iteration:
[40]	train-merror:0.257145	valid-merror:0.263901	train--kappa:-0.561743	valid--kappa:-0.48533

(60937, 362) (15233, 362)
[0]	train-merror:0.298846	valid-merror:0.289503	train--kappa:-0.490188	valid--kappa:-0.441172
Multiple eval metrics have been passed: 'valid--kappa' will be used for early stopping.

Will train until valid--kappa hasn't improved in 5 rounds.
[1]	train-merror:0.291264	valid-merror:0.280706	train--kappa:-0.503596	valid--kappa:-0.453092
[2]	train-merror:0.287444	valid-merror:0.278

In [176]:
test[feats]

,bef_label,I1,I2,I3_x,I5,I6,I11,I13,I14,I16,...,E8,E9,E10,E12,E13,E14,E15,E16,E18,event_num
0,0.0,0,2.0,3,7,0,0.0,2,5,1.0,...,3,3,3,3,3,3,1170000.00,3,3,13
1,1.0,0,4.0,3,0,1,0.0,2,5,1.0,...,3,3,3,3,3,3,5219392.44,3,3,13
2,0.0,1,7.0,3,0,0,0.0,2,5,1.0,...,3,3,3,3,3,3,0.00,3,3,6
3,0.0,0,4.0,3,0,0,0.0,2,5,1.0,...,3,3,3,3,3,1,298000.00,3,3,11
4,0.0,0,6.0,3,0,0,0.0,2,5,1.0,...,3,3,3,3,3,3,1700000.00,3,3,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76717,0.0,0,1.0,0,5,0,0.0,2,5,0.0,...,3,3,3,3,3,3,50000.00,2,2,12
76718,1.0,0,6.0,3,7,0,0.0,2,5,1.0,...,3,3,1,3,3,3,0.00,3,3,4
76719,1.0,0,5.0,1,0,0,0.0,2,5,1.0,...,3,3,3,3,3,3,50000.00,3,2,11
76720,1.0,1,7.0,0,3,0,0.0,2,5,0.0,...,3,3,3,3,3,3,50000.00,3,3,9


In [228]:
xgb_preds

[array([[0.2100434 , 0.4403957 , 0.34956092],
        [0.18490225, 0.25873026, 0.5563675 ],
        [0.20797056, 0.27410734, 0.5179221 ],
        ...,
        [0.20065783, 0.22644511, 0.5728971 ],
        [0.20172484, 0.23121604, 0.5670591 ],
        [0.25281537, 0.43034598, 0.31683865]], dtype=float32),
 array([[0.17670494, 0.46425682, 0.3590383 ],
        [0.16864805, 0.22110534, 0.61024666],
        [0.18505928, 0.24781138, 0.5671294 ],
        ...,
        [0.19121239, 0.2235194 , 0.5852682 ],
        [0.19120592, 0.21455933, 0.59423476],
        [0.2322962 , 0.46891284, 0.2987909 ]], dtype=float32),
 array([[0.11662882, 0.5135601 , 0.36981103],
        [0.08762727, 0.19194531, 0.72042745],
        [0.11479064, 0.22645162, 0.65875775],
        ...,
        [0.10923067, 0.22791663, 0.6628527 ],
        [0.11030485, 0.1597722 , 0.72992295],
        [0.17216912, 0.5700958 , 0.25773513]], dtype=float32),
 array([[0.18073107, 0.46815312, 0.3511158 ],
        [0.17180707, 0.2312654 , 0.5

In [229]:
sub_df = test[['cust_no']].copy()
sub_df['label'] = np.argmax(np.mean(xgb_preds, axis=0), axis=1) - 1
sub_df['label'].value_counts(normalize=True)

 1    0.636141
 0    0.237051
-1    0.126808
Name: label, dtype: float64

In [234]:
sub_df[['prba1', 'prba2', 'prba3']] = np.mean(xgb_preds, axis=0)
sub_df

,cust_no,label,prba1,prba2,prba3
0,0x3b9b4615,0,0.166876,0.472870,0.360254
1,0x3b9ae61b,1,0.147090,0.219642,0.633269
2,0x3b9add69,1,0.167923,0.252819,0.579258
3,0x3b9b3601,0,0.141970,0.532822,0.325208
4,0x3b9b2599,0,0.135681,0.445967,0.418352
...,...,...,...,...,...
76717,0xb2d69017,1,0.216991,0.298102,0.484907
76718,0xb2d68153,1,0.128733,0.226702,0.644565
76719,0xb2d5bba1,1,0.159779,0.223297,0.616924
76720,0xb2d61b9b,1,0.165601,0.197916,0.636483


In [235]:
sub_df.to_csv('xgb_1_prab.csv', index=False)


In [230]:
sub_df.to_csv('xgb_1.csv', index=False)


In [232]:
(ensemble_1['label']==xgb_1['label']).value_counts()

True     72478
False     4244
Name: label, dtype: int64

In [233]:
(lgb_485['label']==xgb_1['label']).value_counts()

True     73202
False     3520
Name: label, dtype: int64

In [241]:
ensemble_2 = pd.DataFrame(cnn_445['cust_no'])
ensemble_2['label'] =  np.argmax((0.45*lgb_485[['prba1', 'prba2', 'prba3']].values + 0.3*xgb_1_prab[['prba1', 'prba2', 'prba3']].values  + 0.25*cnn_445[['pred_-1', 'pred_0', 'pred_1']].values), axis=1) - 1
ensemble_2

,cust_no,label
0,0x3b9b4615,0
1,0x3b9ae61b,1
2,0x3b9add69,1
3,0x3b9b3601,0
4,0x3b9b2599,1
...,...,...
76717,0xb2d69017,1
76718,0xb2d68153,1
76719,0xb2d5bba1,1
76720,0xb2d61b9b,1


In [242]:
(lgb_485['label']==ensemble_2['label']).value_counts()

True     73983
False     2739
Name: label, dtype: int64

In [243]:
(ensemble_1['label']==ensemble_2['label']).value_counts()

True     75954
False      768
Name: label, dtype: int64

In [244]:
ensemble_2.to_csv('ensemble_2.csv', index=False)
